## ` Multy Agent System & Workflow Patterns `

#### Instead of one "do-it-all" agent, we can build a multi agent system. This team of simple, specialized agents that collaborate, just like a real world team. Each Agent here has its own job, the first Agent is for Researching given the google_search tool and the second just from its training for creating summarys. The Third one is the coordinator who combines and orchestrates the two other Agents to create the prompts outcome.

In [18]:
import os
try:
  GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
  print("Gemini API key setup complete.")
except Exception as e:
  print(f"Failed : {e}")

Gemini API key setup complete.


In [19]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print(" ADK components imported successfully.")

 ADK components imported successfully.


In [20]:
# Research Agent: Its job is to use the google_search tool and present findings.
research_agent = Agent(
  name="ResearchAgent",
  model="gemini-2.5-flash-lite",
  instruction="""You are a specialized research agent. Your only job is to use the
    google_search tool to find 2-3 pieces of relevant information on the given topic and present the findings with citations.""",
  tools=[google_search],
  output_key="research_findings"
)

print("research_agent has created.")

research_agent has created.


In [21]:
# Summarize Agent : Its job is to summarize the text that its given.

summarize_agent = Agent(
  name="SummarizeAgent",
  model="gemini-2.5-flash-lite",
  instruction="""Read the provided research findings: {research_findings}
    Create a concise summary as a bulleted list with 3-5 key points.""",
  tools=[],
  output_key="final summary"
)
print("summarize_agent has created.")

summarize_agent has created.


In [22]:
# Root Coordinator: Orchestrates the workflow by calling the sub-agents as tools.

root_agent = Agent(
  name = "CoordinatorAgent",
  model="gemini-2.5-flash-lite",
  instruction="""You are a research coordinator. Your goal is to answer the user's query by orchestrating a workflow.
1. First, you MUST call the `ResearchAgent` tool to find relevant information on the topic provided by the user.
2. Next, after receiving the research findings, you MUST call the `SummarizerAgent` tool to create a concise summary.
3. Finally, present the final summary clearly to the user as your response.""",
  tools=[
    AgentTool(research_agent),
    AgentTool(summarize_agent)
  ]
)

print("Root Agent created.")

Root Agent created.


In [23]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug("What are the latest advancements in quantum computing and what do they mean for AI?")


 ### Created new session: debug_session_id

User > What are the latest advancements in quantum computing and what do they mean for AI?


CoordinatorAgent > Quantum AI, the integration of quantum computing and artificial intelligence, is poised to revolutionize industries by enabling AI to tackle complex problems currently insurmountable for classical computers. This synergy promises exponential performance gains, significantly faster AI model training, enhanced optimization capabilities, and more efficient data processing.

Key advancements and implications include:

*   **Overcoming Classical Limitations:** Quantum computers' ability to process vast amounts of information simultaneously addresses the immense data and power requirements of training deep learning models, a limitation of classical AI.
*   **Accelerated AI Development:** Quantum machine learning algorithms can dramatically speed up AI learning and adaptation, with potential for substantial energy savings. Advancements in quantum hardware are crucial for realizing this potential.
*   **New AI Capabilities:** Quantum AI could enable AI to achieve new levels 

## **The Problem: `Unpredictable Order`**
### Sequential Agent for linear problems

The previous multi-agent system worked, but it relied on a **detailed instruction prompt** to force the LLM to run steps in order. This can be unreliable. A complex LLM might decide to skip a step, run them in the wrong order, or get "stuck," making the process unpredictable.

**The Solution: A Fixed Pipeline**

When you need tasks to happen in a **guaranteed, specific order**, you can use a `SequentialAgent`. This agent acts like an assembly line, running each sub-agent in the exact order you list them. The output of one agent automatically becomes the input for the next, creating a predictable and reliable workflow.


**Use Sequential when:** Order matters, you need a linear pipeline, or each step builds on the previous one.

### Blog Post Creation with Sequential Agents
1. **Outline Agent** : Creates a blog outline for a given topic.
2. **Writer Agent** : Writes a blog post
3. **Editor Agent** : Edits a blog post draft for clarity and structure

In [31]:
# Creates the initial blog post outline.
outline_agent = Agent(
  name="OutlineAgent",
  model="gemini-2.5-flash-lite",
  instruction="""Create a blog outline for the given topic with:
    1. A catchy headline
    2. An introduction hook
    3. 3-5 main sections with 2-3 bullet points for each
    4. A concluding thought""",
  output_key="blog_outline" ## The result of this agent will be stored in the session state with this key.
)
print("Outline Agent has created.")

Outline Agent has created.


In [32]:
# Writes the full blog post based on the outline from the previous agent.
writer_agent = Agent(
  name="WriterAgent",
  model="gemini-2.5-flash-lite",
  instruction="""Following this outline strictly: {blog_outline}
    Write a brief, 200 to 300-word blog post with an engaging and informative tone.""",
  output_key="blog_draft"
)
print("Writer Agent has created.")

Writer Agent has created.


In [33]:
editor_agent = Agent(
  name="EditorAgent",
  model="gemini-2.5-flash-lite",
  instruction="""Edit this draft: {blog_draft}
    Your task is to polish the text by fixing any grammatical errors, improving the flow and sentence structure, and enhancing overall clarity.""",

)
print("Editor Agent created")

Editor Agent created


In [34]:
root_agent = SequentialAgent(
  name="BlogPipeline",
  sub_agents=[outline_agent,writer_agent,editor_agent],

)

print("SequentialAgent has created.")

SequentialAgent has created.


In [35]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug("Write a blog post about the benefits of multi-agent systems for software developers.")


 ### Created new session: debug_session_id

User > Write a blog post about the benefits of multi-agent systems for software developers.
OutlineAgent > Okay, I am OutlineAgent. Here is a blog outline about the benefits of multi-agent systems for software developers:

## Blog Outline: Multi-Agent Systems for Software Developers

**Catchy Headline:** Level Up Your Code: How Multi-Agent Systems are Revolutionizing Software Development

**Introduction Hook:**
Imagine a development team where each member is an autonomous, specialized expert, communicating seamlessly to tackle complex tasks. This isn't a sci-fi dream; it's the reality of multi-agent systems (MAS), and they're poised to transform how software developers build, test, and deploy. If you're looking for ways to boost efficiency, handle complexity, and unlock new levels of innovation, it's time to explore the power of MAS.

---

**Main Sections:**

**1. Enhanced Modularity and Maintainability**
    *   **Decomposition into Indepen

###  `Parallel Workflows - Indepedent Researchers` 

The previous sequential agent is great, but it's a line. Each step must wait for the previous one to finish. What if you have several tasks that are **not dependent** on each other? For example, researching three *different* topics. Running them in sequence would be slow and inefficient, creating a bottleneck where each task waits unnecessarily.

**The Solution: `Concurrent Execution`**
When we have indepedent tasks, we can run all at the same time using a 'ParallelAgent'. This agent executes all of its sub-agents concurrently, dramatically speeding up the workflow. Once All parallel tasks are comple, you can then pass their combined results to a final 'aggregaror' step.

**Use Parallel when:** Tasks are independent, speed matters, and you can execute concurrently.



### Pararell Multi-Topic Research

Lets build a system with four agents:

1. **Tech Researcher** : Researches AI/ML news and trends.
2. **Health Researcher** : Researches recent medical new and trends.
3. **Finance Researcher** : Researches finance and fintech news and trends.
4. **Aggregator Agent**: Combines all research findings into a single summary.

In [37]:
# Focuses on AI and ML trends.

tech_researcher = Agent(
  name="TechResearcher",
  model="gemini-2.5-flash-lite",
  instruction="""Research the latest AI,ML trends. Include 3 key developments,
   the main companies involved, and the potential impact. Keep the report very concise(100 words). """,
   tools=[google_search],
   output_key="tech_research"
)
print("Tech Researcher agent has created.")

Tech Researcher agent has created.


In [39]:
# Health Researcher: Focuses on medical breakthroughs.
health_researcher = Agent(
  name="HealthResearcher",
  model="gemini-2.5-flash-lite",
  instruction= """Research recent medical breakthroughs. Include 3 significant advances,
their practical applications, and estimated timelines. Keep the report concise (100 words).""",
  tools=[google_search],
  output_key="health_research"
)
print("Health Researcher agent has created.")

Health Researcher agent has created.


In [40]:
# Focuses on fintech trends.
finance_researcher = Agent(
  name="FinanceResearcher",
  model="gemini-2.5-flash-lite",
  instruction= """Research current fintech trends. Include 3 key trends,
their market implications, and the future outlook. Keep the report concise (100 words).""",
  tools=[google_search],
  output_key="finance_research"
)
print("Finance Researcher agent has created.")

Finance Researcher agent has created.


In [41]:
aggregator_agent = Agent(
  name="AggregatorAgent",
  model="gemini-2.5-flash-lite",
  instruction="""Combine these three research findings into a single executive summary:

    **Technology Trends:**
    {tech_research}

    **Health Breakthroughs:**
    {health_research}

    **Finance Innovations:**
    {finance_research}

    Your summary should highlight common themes,
    surprising connections, and the most important key takeaways
    from all three reports. The final summary should be around 200 words.""",
    output_key="executive_summary"
)
print("Aggregator Agent created")

Aggregator Agent created


In [42]:
# The parallelAgent runs all its sub-agents simultaneously.
parallel_research_team = ParallelAgent(
  name="ParallelResearchTeam",
  sub_agents=[tech_researcher,health_researcher,finance_researcher]
)
print("parallel_research_team Agent has created.")
#This SequentialAgent defines the high level workflow: run the parallel team first,the run the aggregator.
root_agent = SequentialAgent(
  name="ResearchSystem",
  sub_agents=[parallel_research_team,aggregator_agent]
)
print("Root Agent has created.")

parallel_research_team Agent has created.
Root Agent has created.


In [43]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug("Run the daily executive briefing on Tech, Health and finance.")


 ### Created new session: debug_session_id

User > Run the daily executive briefing on Tech, Health and finance.
FinanceResearcher > **Tech Trends:**

1.  **AI Workloads & Infrastructure Orchestration:** The demand for AI, especially large language models, is driving significant infrastructure investment and a shift towards sophisticated orchestration strategies using tools like Kubernetes to manage GPU resources efficiently.
2.  **Agentic AI & Context Engineering:** AI is evolving from simple chatbots to autonomous agents capable of complex tasks. This necessitates rigorous "context engineering" – structuring data for AI to ensure reliable performance.
3.  **Hardware Advancements:** The tech industry sees a surge in AI-powered laptops with dedicated AI processors, and advancements in chip manufacturing like Samsung's 2nm process promise cooler, more efficient performance.

**Health Trends:**

1.  **Personalized Healthcare & AI:** Healthcare is becoming increasingly personalized, leve

###  `Loop Workflows - The Refinement Cycle` 

All the workflows so far run from start to finish. Both SequentialAgent and ParallelAgent produce their final output and the stop. This 'one-shot' approach isnt good for tasks that require refinement and quality control.

**The Solution: `Iterative Refinment`**

When a task need to be improved through cycles of feedback and revision, we cant use `LoopAgent`, It runs a set of sub-agents repeatedly until a specific condition is met or a maximum number of iterations is reached. This creates a refinement cycle, allowing the agent system to improve its own work over and over.

**Use Loop when:** Iterative improvement is needed, quality refinement matters or you need repeated cycles.



### Iterative Story Refinement

1. **Writer Agent** : Writes a draft of a short story
2. **Critic Agent** :Reviews and critiques the short story to suggest improvements

In [44]:
# runs once at the beggining to create the first draft.
initial_writer_agent = Agent(
  name="InitialWriterAgent",
  model="gemini-2.5-flash-lite",
  instruction="""Based on the user's prompt, write the first draft of a short story (around 100-150 words).
    Output only the story text, with no introduction or explanation.""",
  output_key="current_story"

)
print("initial_writer_agent created.")

initial_writer_agent created.


In [45]:
# provide feedback or the approval signal.
critic_agent=Agent(
  name="CriticAgent",
  model="gemini-2.5-flash-lite",
  instruction="""You are a constructive story critic. Review the story provided below.
    Story: {current_story}

    Evaluate the story's plot, characters, and pacing.
    - If the story is well-written and complete, you MUST respond with the exact phrase: "APPROVED"
    - Otherwise, provide 2-3 specific, actionable suggestions for improvement.""",
  output_key="critique"
)
print("critic_agent has created.")

critic_agent has created.


Now, we need a way for the loop to actually stop based on the critic's feedback. The `LoopAgent` itself doesn't automatically know that "APPROVED" means "stop."

We need an agent to give it an explicit signal to terminate the loop.

We do this in two parts:

1. A simple Python function that the `LoopAgent` understands as an "exit" signal.
2. An agent that can call that function when the right condition is met.

In [48]:
def exit_loop():
  """Call this function only when the critique is 'APPROVED',indicating the story is finished and no more changes are needed."""
  return {"status":"approved","message":"Story approved. Exiting refinement loop."}

print("exit_loop function has created.")

exit_loop function has created.


In [49]:
# refines the story based on critique or calls the exit_loop function

refiner_agent=Agent(
  name="RefinerAgent",
  model="gemini-2.5-flash-lite",
  instruction="""You are a story refiner. You have a story draft and critique.
  Story Draft: {current_story}
  Critique: {critique}

  Your task is to analyze the critique.
  -If the critique is exactly "APPROVED", you must call the "exit_loop" function and
  nothing else. OTHERWISE, rewrite the story draft to fully incorporate the feedback
  from the critique.
  """,
  output_key="current_story",
  tools=[FunctionTool(exit_loop)]
)

print("refiner_agent has created.")

refiner_agent has created.


In [50]:
# the LoopAgent contains the agents that will run repeatedly: critic -> refiner.

story_refinement_loop=LoopAgent(
  name="StoryRefinementLoop",
  sub_agents=[critic_agent,refiner_agent],
  max_iterations=3, #preventing infinite loops
)

print(" Loop Agent created.")

# The root agent is a SequentialAgent that defines the overall workflow: Initial Write -> Refinement Loop
root_agent = SequentialAgent(
  name="StoryPipeline",
  sub_agents=[initial_writer_agent,story_refinement_loop]
)

print(" Sequential Agent created.")

 Loop Agent created.
 Sequential Agent created.


In [51]:
runner = InMemoryRunner(agent=root_agent)
response=await runner.run_debug("Write a short story about a pirate that discovers a mysterious barrel of rum.")


 ### Created new session: debug_session_id

User > Write a short story about a pirate that discovers a mysterious barrel of rum.
InitialWriterAgent > Barnacle Bill, a pirate with a beard like a tangled fishing net and eyes as sharp as a kraken’s beak, stumbled upon it during a raid on a merchant vessel. It was no ordinary cask, this one. Dark wood, no markings, and it hummed with a faint, unsettling energy. He pried it open, the salty air thick with the promise of adventure.

The rum within wasn't the usual golden hue. It shimmered with an unnatural, violet light. Tentatively, Bill took a swig. The taste was unlike anything he’d ever experienced – a blend of starlight, sea mist, and a whisper of forgotten magic. Suddenly, the deck beneath him seemed to warp, the stars in the sky pulsed brighter, and a deep, resonant voice echoed in his mind, "The sea remembers, Barnacle Bill." He knew then, this wasn't just rum; it was a key.
CriticAgent > Here's a review of your story:

The story has

CriticAgent > Excellent! I'm glad we could reach an approved state for the story. It has a compelling premise and a great sense of mystery.
